In [ ]:
### Delete duplicate image assets within collections ###
import ee, re
from collections import Counter
ee.Initialize()

# CHANGE HERE
root_name = "projects/p79amhara3/assets/PS_imgs/79_Amhara_3/" 
#root_name = "projects/yieldpredug/assets/images/"
unique_matches = set()
# CHANGE HERE
locs =['lon10_959lat36_5095', 'lon10_9819lat39_2804', 'lon11_232lat39_0631', 'lon11_2599lat36_8433', 'lon11_2756lat39_545', 'lon12_4805lat37_4768', 'lon12_5496lat37_0654', 'lon9_8471lat39_7592', 'lon11_7375lat39_0025', 'lon11_8429lat39_7398', 'lon11_8729lat39_2944', 'lon10_5993lat39_9215', 'lon12_1201lat38_0486', 'lon9_9101lat40_0158', 'lon11_3993lat38_2379', 'lon11_4144lat37_9601', 'lon11_602lat37_946', 'lon12_1651lat38_8403', 'lon12_4289lat38_7151', 'lon11_4581lat38_9063']
toApp=[]
for loc in locs:
    folder_name = root_name +loc
    asset_list = ee.data.getList({'id': folder_name})
    id_list = [item['id'] for item in asset_list]
    for asset in id_list:
        assetID = asset.replace(folder_name, "").replace('/','')
        match = re.search(r'^(.*?)_Analytic', assetID)
        if match:
            match_str = match.group(1)
            unique_matches.add(match_str)
            toApp.append(match_str)

    duplicates = [match for match in list(unique_matches) if sum(1 for id_item in id_list if match in id_item) > 1]
    # Print the duplicates
    print("Duplicates of the string before '_Analytic':", len(duplicates), loc)    
    for duplicate in sorted(duplicates):
        # list all assets with this ID
        result = [item for item in id_list if duplicate in item]
        for res in result[1:]: # delete the second result and onwards
            ee.data.deleteAsset(res)
            #print(f'deleted asset {res}')

In [ ]:
## Access data on planet orders; paste order ID from planet dashboard
https://api.planet.com/compute/ops/orders/v2/cfd86736-ac14-4cd5-812e-96b8b88de38e

In [ ]:
# create new service account credentials # makes for faster Planet delivery
!base64  "<xxx>_service_account_key.json" > anna_service_account_key_encoded.txt # download from GCP, IAM roles
print('here')
import ee
service_account = '<xxx>-service-account@<gcp name>.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, '<xxx>_service_account_key.json')
ee.Initialize(credentials)


In [ ]:
# print all locs in a locgroup
import json
from google.cloud import storage
storage_client = storage.Client()
bucketName='<BUCKETNAME>'
bucket=storage_client.get_bucket(bucketName)
locGroup = "<LOCGROUP>"
data_string=bucket.get_blob('{}/{}.geojson'.format(locGroup, locGroup)).download_as_string()
data = json.loads(data_string)
locList=[]
for j in data['features']:
    loc = j['properties']['mktID'].replace(".", "_", 2) #fix name format of mktID
    dumped=json.dumps(j)
    added='{"type": "FeatureCollection", "features": [{'+dumped[1:-1]+'}]}'
    locList.append(loc) 
print(locList)
formatted_string = ' '.join(locList)
print(formatted_string)

In [ ]:
## CREATE NEW GCPs ###
from google.cloud import storage
import subprocess, json, time
storage_client = storage.Client()

def apply_iam_policy_with_backoff(project_id):
    for attempt in range(5):
        print(attempt)
        try:
            command = f"gcloud projects set-iam-policy {project_id} ~/iam_policy.json"
            subprocess.run(command, check=True, input="Y\n",  shell=True, capture_output=True, text=True)
            print(f"IAM policy applied successfully to project {project_id}")
            break
        except subprocess.CalledProcessError as e:
            print(f"Error applying IAM policy to project {project_id}. Attempt {attempt + 1}/5. Error message: {e}")
            time.sleep(1.2 ** attempt)
    else:
        print(f"Failed to apply IAM policy to project {project_id} after 5 attempts.")

def createNewGCP(project_id):
    for attempt in range(5):
        print(attempt)
        try:
            command = f"gcloud projects create {project_id} --name={project_id}"
            result =subprocess.run(command, check=True, shell=True,capture_output=True, text=True)
            print(result.stdout)
            print(f"Successfully created GCP {project_id}")
            break
        except subprocess.CalledProcessError as e:
            print(f"Error creating {project_id}. Attempt {attempt + 1}/5. Error message: {e}")
            time.sleep(1.2 ** attempt)
    else:
        print(f"Failed to create {project_id} after 5 attempts.")

from google.cloud import storage

def copy_file(source_bucket_name, source_blob_name, destination_blob_name):
    """Copies a file from one GCS bucket to another."""
    bucket = storage_client.bucket(source_bucket_name)
    source_blob = bucket.blob(source_blob_name)
    blob_copy = bucket.copy_blob(source_blob, bucket, destination_blob_name)
    print(f"File '{source_blob_name}' copied from '{source_bucket_name}' to '{bucket}' as '{destination_blob_name}'.")

        
### Usage ###

# If needing to generate a new iam_policy.json, delete the etag and version lines in it
#gcloud projects get-iam-policy <GCP WITH CORRECT IAM ROLES> --format json >~/iam_policy.json


gcpToCreate = ["<<ENTER NAMES>>"]
for gcp in gcpToCreate:
    command = f"gcloud config set project planetupload"
    subprocess.run(command, check=True, shell=True)
    gcpshort= 'p' + gcp.replace('_','',5).lower()
    print(gcp, gcpshort)
    copy_file('mai_2023', f'geojsonsToPutIntoFolders/{gcp}.geojson',
              f'{gcp}/{gcp}.geojson')
    createNewGCP(gcpshort)
    apply_iam_policy_with_backoff(gcpshort)
    command = f"gcloud config set project {gcpshort}"
    subprocess.run(command, check=True, shell=True)
    command = f"gcloud services enable earthengine.googleapis.com"
    subprocess.run(command, check=True, shell=True)

print('done')

# Still need to click thru in GEE to register each project

# (1) Create folder with name locGroup in planetupload/mai_2023
# (2) Put geojson file of locGroup in that folder
# (3) Create a new cloud project with the name lcoGroup
# (4) Add that project to GEE
# (5) Add 'planet-gee-uploader@planet-earthengine-staging.iam.gserviceaccount.com' as Earth Engine Resource Admin and Storage Admin in GCP 
# (6) create Master location file in 01_controller.ipynb
# (7) Monitor planet orders to see whether they deliver correctly

In [ ]:
# CREATE NEW MASTER LOCATION FILE ###
import json, csv #, dropbox
from google.cloud import storage

bucketName='<BUCKET NAME>'

storage_client = storage.Client()
bucket=storage_client.get_bucket(bucketName)

### List here all location groups ###
locGroups = ['<LOCGROUP>']
#####################################
for locGroup in locGroups:
    with open('../masterLocationFile{}.csv'.format(locGroup), 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)

        # Write the header row
        csvwriter.writerow(['locGroup', 'Location', "bucket", "00DownStatus", "00aDownNoSRStatus","01PrepStarted","01aPrepComplete","01bPrepFailed","02MapStarted","02aMapComplete","02bMapFailed","03ActStarted","03aActComplete","03bActFailed","04PropertiesDownload"])

        data_string = bucket.get_blob('{}/{}.geojson'.format(locGroup, locGroup)).download_as_string()
        data_json = json.loads(data_string)
        cc=0
        for j in data_json['features']:
            cc+=1
            loc = j['properties']['mktID'].replace(".", "_", 2)
            csvwriter.writerow([locGroup, loc,locGroup," "," "," "," "," "," "," "," "," "," "," "," "])
                
print('done')